In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

In [2]:
# read the file
df = pd.read_csv('groww_mutual_funds.csv')

In [3]:
df

,fund_name,risk,category,rating,1y_return,3y_return,5y_return
0,HDFC Silver ETF FoF Direct Growth,Very High Risk,Commodities,NaN,159.93%,53.58%,NaN
1,SBI Gold Direct Plan Growth,Very High Risk,Commodities,4.0,78.67%,37.57%,25.18%
2,Parag Parikh Flexi Cap Fund Direct Growth,Very High Risk,Equity,5.0,6.89%,20.72%,18.76%
3,Bandhan Small Cap Fund Direct Growth,Very High Risk,Equity,5.0,9.51%,32.50%,25.94%
4,Motilal Oswal Midcap Fund Direct Growth,Very High Risk,Equity,3.0,-1.04%,24.93%,26.08%
...,...,...,...,...,...,...,...
1638,HSBC Gilt Fund Direct Growth,Moderate Risk,Debt,2.0,2.88%,6.42%,5.33%
1639,Bandhan Crisil IBX Gilt April 2032 Index Fund ...,Moderate Risk,Debt,NaN,7.06%,NaN,NaN
1640,HSBC Money Market Fund Direct Growth,Low to Moderate Risk,Debt,3.0,7.20%,7.45%,6.12%
1641,Invesco India Gilt Fund Direct Growth,Moderate Risk,Debt,3.0,2.99%,7.05%,5.53%


In [4]:
# info of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1643 entries, 0 to 1642
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fund_name  1643 non-null   object 
 1   risk       1643 non-null   object 
 2   category   1630 non-null   object 
 3   rating     919 non-null    float64
 4   1y_return  1440 non-null   object 
 5   3y_return  1112 non-null   object 
 6   5y_return  862 non-null    object 
dtypes: float64(1), object(6)
memory usage: 90.0+ KB


- Total columns - 7
    - Numerical columns: 1
    - Categorical columns: 6

In [5]:
# shape of the dataset (columns and rows)
print(f'Shape: {df.shape}')

Shape: (1643, 7)


In [6]:
# statastics of the data
df.describe()

,rating
count,919.000000
mean,3.014146
std,1.101997
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [7]:
# all the column names
df.columns

Index(['fund_name', 'risk', 'category', 'rating', '1y_return', '3y_return',
       '5y_return'],
      dtype='object')

In [8]:
# check for the null values
df.isnull().sum()

fund_name      0
risk           0
category      13
rating       724
1y_return    203
3y_return    531
5y_return    781
dtype: int64

In [9]:
missing = df.isnull().sum()
missing[missing > 0]

category      13
rating       724
1y_return    203
3y_return    531
5y_return    781
dtype: int64

In [10]:
# drop null values based on column 'fund_name', '1y_return'
df = df.dropna(subset=['category', '1y_return'])

# filling the null values of the column 'rating' with the median to keep the distribution realistic
df['rating'] = df['rating'].fillna(df['rating'].median())

# check for any null values
df.isnull().sum()

fund_name      0
risk           0
category       0
rating         0
1y_return      0
3y_return    326
5y_return    576
dtype: int64

- Few new funds often lack 1, 3 and 5-year histories, fill them with 0, creating bias, so do not drop their null values
- Drop '1y-return' funds with null values, as we cannot determine without at least 1 year returns
- Median used instead of mean because financial metrics are often skewed.

In [11]:
df.duplicated().sum()

np.int64(8)

In [12]:
df = df.drop_duplicates(subset=["fund_name"])

In [13]:
df.duplicated().sum()

np.int64(0)

In [14]:
# convert the returns from string to numeric.
for col in ['1y_return', '3y_return', '5y_return']:
    df[col] = df[col].str.replace("%", "", regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [15]:
# verify the conversion
df.dtypes

fund_name     object
risk          object
category      object
rating       float64
1y_return    float64
3y_return    float64
5y_return    float64
dtype: object

In [16]:
# Remove hidden spaces from scraping
df["fund_name"] = df["fund_name"].str.strip()
df["risk"] = df["risk"].str.strip()
df["category"] = df["category"].str.strip()

In [17]:
bins = [-100, 0, 5, 10, 15, np.inf]
labels = ["Loss", "Low", "Moderate", "Good", "High"]

for col in ["1y_return", "3y_return", "5y_return"]:
    band_col = col + "_band"
    df[band_col] = pd.cut(df[col], bins=bins, labels=labels)
    df[band_col] = df[band_col].cat.add_categories("No Data").fillna("No Data")
df

,fund_name,risk,category,rating,1y_return,3y_return,5y_return,1y_return_band,3y_return_band,5y_return_band
0,HDFC Silver ETF FoF Direct Growth,Very High Risk,Commodities,3.0,159.93,53.58,NaN,High,High,No Data
1,SBI Gold Direct Plan Growth,Very High Risk,Commodities,4.0,78.67,37.57,25.18,High,High,High
2,Parag Parikh Flexi Cap Fund Direct Growth,Very High Risk,Equity,5.0,6.89,20.72,18.76,Moderate,High,High
3,Bandhan Small Cap Fund Direct Growth,Very High Risk,Equity,5.0,9.51,32.50,25.94,Moderate,High,High
4,Motilal Oswal Midcap Fund Direct Growth,Very High Risk,Equity,3.0,-1.04,24.93,26.08,Loss,High,High
...,...,...,...,...,...,...,...,...,...,...
1638,HSBC Gilt Fund Direct Growth,Moderate Risk,Debt,2.0,2.88,6.42,5.33,Low,Moderate,Moderate
1639,Bandhan Crisil IBX Gilt April 2032 Index Fund ...,Moderate Risk,Debt,3.0,7.06,NaN,NaN,Moderate,No Data,No Data
1640,HSBC Money Market Fund Direct Growth,Low to Moderate Risk,Debt,3.0,7.20,7.45,6.12,Moderate,Moderate,Moderate
1641,Invesco India Gilt Fund Direct Growth,Moderate Risk,Debt,3.0,2.99,7.05,5.53,Low,Moderate,Moderate


In [18]:
df.describe()

,rating,1y_return,3y_return,5y_return
count,1429.000000,1429.000000,1104.000000,855.000000
mean,3.010497,10.806543,14.582201,12.183310
std,0.878805,16.246895,7.490306,6.053059
min,1.000000,-15.760000,3.830000,-1.140000
25%,3.000000,6.170000,7.690000,6.390000
50%,3.000000,7.580000,14.095000,11.510000
75%,3.000000,11.010000,19.495000,16.300000
max,5.000000,164.160000,53.630000,32.780000


In [19]:
print(df["1y_return_band"].value_counts())
print('--'*15)
print(df["3y_return_band"].value_counts())
print('--'*15)
print(df["5y_return_band"].value_counts())
print('--'*15)

1y_return_band
Moderate    729
Good        274
High        188
Low         178
Loss         60
No Data       0
Name: count, dtype: int64
------------------------------
3y_return_band
High        497
Moderate    433
No Data     325
Good        171
Low           3
Loss          0
Name: count, dtype: int64
------------------------------
5y_return_band
No Data     574
Moderate    361
High        275
Good        209
Low           9
Loss          1
Name: count, dtype: int64
------------------------------


In [20]:
df['risk'].unique()

array(['Very High Risk', 'High Risk', 'Moderately High Risk',
       'Moderate Risk', 'Low to Moderate Risk', 'Low Risk',
       'Moderately Low Risk'], dtype=object)

In [21]:
# Gives a single performance metric across time horizons
df["avg_return"] = df[["1y_return","3y_return","5y_return"]].mean(axis=1)

# Convert categorical risk into a numeric scale
risk_map = {
    "Low Risk": 1,
    "Moderately Low Risk": 2,
    "Moderate Risk": 3,
    "Low to Moderate Risk": 4,
    "Moderately High Risk": 5,
    "High Risk": 6,
    "Very High Risk": 7
}
df["risk_score"] = df["risk"].map(risk_map)

# Risk Adjusted Return
df["risk_adjusted_return"] = df["avg_return"] / df["risk_score"]

# Measures how consistent returns are across years (Lower value → more stable performance, Higher value → volatile returns)
df["return_std"] = df[["1y_return", "3y_return", "5y_return"]].std(axis=1)

# fund age
df['fund_age'] = df[['1y_return','3y_return','5y_return']].notnull().sum(axis=1)
df[df['fund_age'] == 3]['avg_return'].mean()

# Label the Age Groups
def age_label(x):
    if x == 1:
        return "New (≤1Y history)"
    elif x == 2:
        return "Mid-age (≤3Y history)"
    else:
        return "Mature (≥5Y history)"

df['fund_age_group'] = df['fund_age_available'].apply(age_label)


df

,fund_name,risk,category,rating,1y_return,3y_return,5y_return,1y_return_band,3y_return_band,5y_return_band,avg_return,risk_score,risk_adjusted_return,return_std,fund_age
0,HDFC Silver ETF FoF Direct Growth,Very High Risk,Commodities,3.0,159.93,53.58,NaN,High,High,No Data,106.755000,7,15.250714,75.200806,2
1,SBI Gold Direct Plan Growth,Very High Risk,Commodities,4.0,78.67,37.57,25.18,High,High,High,47.140000,7,6.734286,27.999709,3
2,Parag Parikh Flexi Cap Fund Direct Growth,Very High Risk,Equity,5.0,6.89,20.72,18.76,Moderate,High,High,15.456667,7,2.208095,7.483397,3
3,Bandhan Small Cap Fund Direct Growth,Very High Risk,Equity,5.0,9.51,32.50,25.94,Moderate,High,High,22.650000,7,3.235714,11.842850,3
4,Motilal Oswal Midcap Fund Direct Growth,Very High Risk,Equity,3.0,-1.04,24.93,26.08,Loss,High,High,16.656667,7,2.379524,15.336546,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,HSBC Gilt Fund Direct Growth,Moderate Risk,Debt,2.0,2.88,6.42,5.33,Low,Moderate,Moderate,4.876667,3,1.625556,1.813018,3
1639,Bandhan Crisil IBX Gilt April 2032 Index Fund ...,Moderate Risk,Debt,3.0,7.06,NaN,NaN,Moderate,No Data,No Data,7.060000,3,2.353333,NaN,1
1640,HSBC Money Market Fund Direct Growth,Low to Moderate Risk,Debt,3.0,7.20,7.45,6.12,Moderate,Moderate,Moderate,6.923333,4,1.730833,0.706847,3
1641,Invesco India Gilt Fund Direct Growth,Moderate Risk,Debt,3.0,2.99,7.05,5.53,Low,Moderate,Moderate,5.190000,3,1.730000,2.051244,3


In [22]:
df.isnull().sum()

fund_name                 0
risk                      0
category                  0
rating                    0
1y_return                 0
3y_return               325
5y_return               574
1y_return_band            0
3y_return_band            0
5y_return_band            0
avg_return                0
risk_score                0
risk_adjusted_return      0
return_std              325
fund_age                  0
dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1429 entries, 0 to 1642
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   fund_name             1429 non-null   object  
 1   risk                  1429 non-null   object  
 2   category              1429 non-null   object  
 3   rating                1429 non-null   float64 
 4   1y_return             1429 non-null   float64 
 5   3y_return             1104 non-null   float64 
 6   5y_return             855 non-null    float64 
 7   1y_return_band        1429 non-null   category
 8   3y_return_band        1429 non-null   category
 9   5y_return_band        1429 non-null   category
 10  avg_return            1429 non-null   float64 
 11  risk_score            1429 non-null   int64   
 12  risk_adjusted_return  1429 non-null   float64 
 13  return_std            1104 non-null   float64 
 14  fund_age              1429 non-null   int64   
dtypes: catego

In [24]:
# Performance Rank
df["rank"] = df["avg_return"].rank(ascending=False)
df

,fund_name,risk,category,rating,1y_return,3y_return,5y_return,1y_return_band,3y_return_band,5y_return_band,avg_return,risk_score,risk_adjusted_return,return_std,fund_age,rank
0,HDFC Silver ETF FoF Direct Growth,Very High Risk,Commodities,3.0,159.93,53.58,NaN,High,High,No Data,106.755000,7,15.250714,75.200806,2,6.0
1,SBI Gold Direct Plan Growth,Very High Risk,Commodities,4.0,78.67,37.57,25.18,High,High,High,47.140000,7,6.734286,27.999709,3,21.0
2,Parag Parikh Flexi Cap Fund Direct Growth,Very High Risk,Equity,5.0,6.89,20.72,18.76,Moderate,High,High,15.456667,7,2.208095,7.483397,3,350.0
3,Bandhan Small Cap Fund Direct Growth,Very High Risk,Equity,5.0,9.51,32.50,25.94,Moderate,High,High,22.650000,7,3.235714,11.842850,3,79.0
4,Motilal Oswal Midcap Fund Direct Growth,Very High Risk,Equity,3.0,-1.04,24.93,26.08,Loss,High,High,16.656667,7,2.379524,15.336546,3,253.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,HSBC Gilt Fund Direct Growth,Moderate Risk,Debt,2.0,2.88,6.42,5.33,Low,Moderate,Moderate,4.876667,3,1.625556,1.813018,3,1333.0
1639,Bandhan Crisil IBX Gilt April 2032 Index Fund ...,Moderate Risk,Debt,3.0,7.06,NaN,NaN,Moderate,No Data,No Data,7.060000,3,2.353333,NaN,1,1045.5
1640,HSBC Money Market Fund Direct Growth,Low to Moderate Risk,Debt,3.0,7.20,7.45,6.12,Moderate,Moderate,Moderate,6.923333,4,1.730833,0.706847,3,1092.0
1641,Invesco India Gilt Fund Direct Growth,Moderate Risk,Debt,3.0,2.99,7.05,5.53,Low,Moderate,Moderate,5.190000,3,1.730000,2.051244,3,1319.0


In [25]:
# Save entire DataFrame to CSV
df.to_csv("groww_cleaned_data.csv", index=False)